<a href="https://colab.research.google.com/github/gd1m3y/Text-Summarizer-Using-Elmo/blob/master/summarizer_using_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
import spacy

In [5]:
import re 
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import time
 

In [18]:
import string
punc = string.punctuation

In [6]:
from spacy.lang.en import English # updated

raw_text = 'Hello, world. Here are two sentences.'
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated
doc = nlp(raw_text)
sentences = [sent.string.strip() for sent in doc.sents]

In [8]:
document = """Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task. Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in the applications of email filtering, detection of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific instructions for performing the task. Machine learning is closely related to computational statistics, which focuses on making predictions using computers. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a field of study within machine learning, and focuses on exploratory data analysis through unsupervised learning.In its application across business problems, machine learning is also referred to as predictive analytics."""


In [9]:
print(document)

Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task. Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in the applications of email filtering, detection of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific instructions for performing the task. Machine learning is closely related to computational statistics, which focuses on making predictions using computers. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a field of study within machine learning, and focuses on exploratory data analysis through unsupervised learning.In its application across business pr

In [10]:
document_2 = """Our Father who art in heaven, hallowed be thy name. Thy kingdom come. Thy will be done, on earth as it is in heaven. Give us this day our daily bread; and forgive us our trespasses, as we forgive those who trespass against us; and lead us not into temptation, but deliver us from evil"""

In [11]:
document_2

'Our Father who art in heaven, hallowed be thy name. Thy kingdom come. Thy will be done, on earth as it is in heaven. Give us this day our daily bread; and forgive us our trespasses, as we forgive those who trespass against us; and lead us not into temptation, but deliver us from evil'

In [103]:
doc = nlp(document)
sentences = [sent.string.strip() for sent in doc.sents]

In [104]:
sentences

['Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task.',
 'Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task.',
 'Machine learning algorithms are used in the applications of email filtering, detection of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific instructions for performing the task.',
 'Machine learning is closely related to computational statistics, which focuses on making predictions using computers.',
 'The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning.',
 'Data mining is a field of study within machine learning, and focuses on exploratory data analysis through unsupervised learning.',
 'In its app

In [105]:
def create_embedding(x):
  #tf hub module
  elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
  embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

  # embeddings.shape
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

def lemmatization(texts):
  output = []
  for i in nlp(texts):
    output.append(i.lemma_)
  output = " ".join(i for i in output)
  return output

def preprocess(x):
  #removing  any links present
  x = re.sub(r'http\S+','',x)
  #removing any punctuation
  x = "".join(i for i in x if i not in punc)
  #removing any numbers 
  x = x.replace('[0-9]'," ")
  #converting all to lower case
  x = x.lower()
  #removing extra white spaces
  x = " ".join(i for i in x.split())

  #lemmaziation / normalization of text 
  x = lemmatization(x)
  return[x]

In [106]:
preprocess(sentences[0]),sentences[0]

(['machine learning ml is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task'],
 'Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task.')

In [107]:
train_data = []

for i in sentences:
  train_data.append(preprocess(i))

In [108]:
train_data

[['machine learning ml is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task'],
 ['machine learning algorithms build a mathematical model of sample data known as training data in order to make predictions or decisions without being explicitly programmed to perform the task'],
 ['machine learning algorithms are used in the applications of email filtering detection of network intruders and computer vision where it is infeasible to develop an algorithm of specific instructions for performing the task'],
 ['machine learning is closely related to computational statistics which focuses on making predictions using computers'],
 ['the study of mathematical optimization delivers methods theory and application domains to the field of machine learning'],
 ['data mining is a field of study within machine learning and focuses on exploratory data analysis through unsupervised learning'],
 ['in its applicat

In [109]:
train_embeddings =[]
for i in train_data:
  train_embeddings.append(create_embedding(i))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [110]:
train_embeddings_array = np.array(train_embeddings)

In [111]:
train_embeddings_array.shape

(7, 1, 1024)

In [112]:
final_embedding = np.concatenate(train_embeddings_array,axis=0)

In [113]:
# train_embeddings
final_embedding.shape

(7, 1024)

In [114]:
from sklearn.cluster import KMeans
n_clusters =int(np.ceil(len(final_embedding)*0.5))

In [115]:
model = KMeans(n_clusters=n_clusters).fit(final_embedding)

In [116]:
model.labels_

array([1, 1, 1, 1, 3, 0, 2], dtype=int32)

In [118]:
len(final_embedding)

7

In [119]:
from sklearn.metrics import pairwise_distances_argmin_min

avg = []
for j in range(n_clusters):
    idx = np.where(model.labels_ == j)[0]
    avg.append(np.mean(idx))
closest, _ = pairwise_distances_argmin_min(model.cluster_centers_, final_embedding)
ordering = sorted(range(n_clusters), key=lambda k: avg[k])
summary = ' '.join([sentences[closest[idx]] for idx in ordering])

In [120]:
summary

'Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a field of study within machine learning, and focuses on exploratory data analysis through unsupervised learning. In its application across business problems, machine learning is also referred to as predictive analytics.'

In [123]:
document

'Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task. Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in the applications of email filtering, detection of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific instructions for performing the task. Machine learning is closely related to computational statistics, which focuses on making predictions using computers. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a field of study within machine learning, and focuses on exploratory data analysis through unsupervised learning.In its application across business p